# Golomovzyi Viktor SA-32

In [2]:
import os
import re
import csv
import pandas as pd

## Dataset Combining

### Data concatenating

In [3]:
def standartize(df, column_map):
    """
    Standartize all column names.

    Args:
        df (DataFrame): DataFrame with raw data.
        column_map (Dict): Dictionary with standartized names.

    Returns:
        DataFrame: DataFrame with standartized column names.
    """
    standartized_columns = {}
    for col in df.columns:
        standartized_columns[col] = column_map.get(col, col)
        
    df.rename(columns = standartized_columns, inplace = True)

    return df

In [4]:
def detect_csv_format(file_path):
    """Detects various separators with built in python csv module.

    Args:
        file_path (str): Path to the DataFrame file.
    
    Returns:
        Dict: Dictionary with specified separator.
    """
    with open(file_path, 'r', encoding = 'utf-8') as file:
        sample = file.read(2048)
        sniffer = csv.Sniffer()
        dialect = sniffer.sniff(sample)

        return{'delimiter': dialect.delimiter, 'quotechar': dialect.quotechar}

In [11]:
def preprocess(in_folder):
    """Mapping, renaming and standartizing columns in files.

    Args:
        in_folder (str): Input folder which contains data to preprocess.
    """

    concatenated_data = []

    column_map = {
        "Зарплата за місяць, net, після податків" : 'Salary',
        "Зарплата в месяц": 'Salary',
        "Зарплата у $$$ за місяць, лише ставка після сплати податків": 'Salary',
        "Зарплата.в.месяц": 'Salary',
        "salary": 'Salary',
        """Зарплата / дохід у $$$ за місяць, лише ставка 
ЧИСТИМИ - після сплати податків""": 'Salary,',
        "Зарплата / дохід у $$$ за місяць, лише ставка після сплати податків": 'Salary',
        """Зарплата / дохід у $$$ за місяць, лише ставка 
ЧИСТИМИ - після сплати податків""": 'Salary',
        """Зарплата / дохід в IT у $$$ за місяць, лише ставка 
ЧИСТИМИ - після сплати податків""": 'Salary',
        """Зарплата / дохід в ІТ у $$$ за місяць, лише ставка 
ЧИСТИМИ - після сплати податків""": 'Salary',


        "Оберіть вашу посаду": 'Position',
        "Должность": 'Position',
        "position": 'Position',
        "Оберіть вашу основну посаду": 'Position',
        "Спеціалізація": 'Position',
        "Position": 'Position',

        "Загальний стаж роботи за спеціальністю": 'Global_exp',
        "Стаж на поточному місці роботи": 'Local_exp',
        "Общий опыт работы": 'Global_exp',
        "Опыт работы на текущем месте": 'Local_exp',
        "exp": 'Global_exp',
        "current_job_exp": 'Local_exp',
        "Загальний стаж роботи в ІТ": 'Global_exp',
        "Загальний стаж роботи за нинішньою спеціалізацією": 'Global_exp',
        "Загальний стаж роботи за нинішньою ІТ-спеціальністю": 'Local_exp',
        "experience": 'Global_exp',
        "Ваш тайтл": 'Seniority',
        "Ваш тайтл на цій посаді": 'Seniority',
        "Оберіть ваш тайтл або роль у компанії": 'Seniority',
    }

    in_files = [file for file in os.listdir(in_folder) if file.endswith('.csv')]
    in_files = sorted(in_files, key = lambda x: int(re.search(r'\d+', x).group()))

    for file in in_files:
        match = re.search(r'(\d{4})_(\w+)_.*\.csv', file)
        year, month = match.groups()
        year = int(year)
        if month.lower() in ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'june']:
            half_year = f"{year}_H1"
        else:
            half_year = f"{year}_H2"
        path = os.path.join(in_folder, file)
        try:
            if half_year in ['2021_H2', '2022_H2', '2023_H1']:
                csv_format = {'delimiter': ';', 'quotechar': '"'}
            else:
                csv_format = detect_csv_format(path)

            df = pd.read_csv(path, on_bad_lines='skip', **csv_format)

            if half_year == "2022_H1":
                df = df[~((df.get("city") == "Всі міста") | (df.get("experience") == 'Весь досвід') | (df.get('position') == 'Всі'))]
            
            df.dropna(how='all', inplace = True)

            df = standartize(df, column_map)

            df['Period'] = half_year
            df = df.loc[:, ~df.columns.duplicated()]
            concatenated_data.append(df)

        except Exception as e:
            print(f"Error processing {file}: {e}")
        continue

    concat_df = pd.concat(concatenated_data, ignore_index = True)
    columns = ['Period', 'Position', 'Seniority', 'Global_exp', 'Local_exp', 'Salary']
    concat_df = concat_df[columns]
    concat_df.to_csv("concat_salary.csv", index = False)

### Data filtering

In [50]:
def filter(in_file):
    df = pd.read_csv(in_file)
    print(df['Position'].unique())
    df = df[df['Position'].str.contains("Business Analyst | Analyst", na = False)].sort_values(by = 'Period').copy()

    df.to_csv('gay_sal.csv', index = False)


In [51]:
def gay(filter):
    print('gay started')
    filter('concat_salary.csv')
    print('gay ended')


if __name__ == '__main__':
    gay(filter)

gay started
['System Architect' 'Designer' 'Senior Project Manager / Program Manager'
 'Software Engineer' 'DevOps' 'Senior Software Engineer' 'Marketing'
 'Product Manager' 'Director of Engineering / Program Director'
 'Technical Lead' 'Team Lead' 'Data Science' 'Research Engineer'
 'Junior Software Engineer' 'Scrum Master' 'HR' 'Security Specialist'
 'Project Manager' 'QA Tech Lead' 'BI Engineer' 'Sysadmin'
 'Senior QA Engineer' 'Business Analyst' 'Sales Manager' 'DBA'
 'Technical writer' 'System Analyst' 'ERP / CRM' 'Support' 'QA Engineer'
 'Data Analyst' 'Copywriter' 'HTML Coder' 'Junior QA' 'QA engineer'
 'Senior QA engineer' 'Project manager' 'Business analyst'
 'Junior QA engineer' 'Data Scientist' 'Team lead' 'HTML coder'
 'Sales manager' 'Game designer' 'Talent Researcher'
 'Software Engineer (Junior, Middle, Senior, Team/Tech Lead, Architect)'
 'Project/Product/Program Manager, Product Owner, Scrum Master, Delivery Manager'
 'Data Science, Machine Learning, Big Data'
 'QA Eng

C:\Users\vgolo\AppData\Local\Temp\ipykernel_12620\197293221.py:2: DtypeWarning: Columns (3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(in_file)


In [12]:
def main(preprocess):
    """Main function to run the preprocess.

    Args:
        preprocess (func): Function that preprocesses the data from files.
    """
    print('Preprocessing files in salary_raw')
    preprocess("salary_raw")
    print('Preprocess ended')


if __name__ == '__main__':
    main(preprocess)

Preprocessing files in salary_raw
Preprocess ended
